# CV + ML Hybrid Pipeline for Error Bar Detection

## Objective
Combine Computer Vision (CV) for coarse detection with Machine Learning (ML) regression for precision refinement.

## Why Hybrid?

### ❌ Pure CV Limitations:
- **Brittle**: Fixed thresholds fail on varied plot styles
- **Noise sensitive**: Edge detection struggles with low contrast
- **No learning**: Can't adapt to dataset patterns

### ❌ Pure ML Limitations:
- **Data hungry**: Requires massive labeled datasets
- **Black box**: No geometric understanding
- **Overfits**: Learns dataset artifacts instead of error bars

### ✅ Hybrid Advantages:
- **CV provides structure**: Coarse geometric priors reduce search space
- **ML refines precision**: Learns to correct CV mistakes from data
- **Data efficient**: ML only needs to learn refinements, not full detection
- **Robust**: CV catches obvious cases, ML fixes edge cases
- **Explainable**: CV stage interpretable, ML stage measurable

## Pipeline Architecture

```
Input Image → CV Stem Detection → Coarse Estimate → ML Regression → Refined Prediction
              (Canny + Projection)  (top_dist, bot_dist)  (delta corrections)  (final pixels)
```

## 1. Setup and Imports

In [1]:
# Install required packages
!pip install opencv-python scikit-learn pandas numpy matplotlib pillow tqdm -q
print("Libraries installed successfully!")

Libraries installed successfully!


In [2]:
import cv2
import numpy as np
import pandas as pd
import json
import os
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Configuration and Paths

In [3]:
# Data paths - Kaggle format
BASE_PATH = "/kaggle/input/graph-plots"
TRAIN_IMAGES = os.path.join(BASE_PATH, "Train", "images")
TRAIN_LABELS = os.path.join(BASE_PATH, "Train", "labels")
TEST_IMAGES = os.path.join(BASE_PATH, "Test", "images")
TEST_INPUT_LABELS = os.path.join(BASE_PATH, "Test", "test_labels")
TEST_GROUND_TRUTH = os.path.join(BASE_PATH, "Test", "labels")

# For local testing, use:
# BASE_PATH = "dataset task"
# TRAIN_IMAGES = os.path.join(BASE_PATH, "images")
# TRAIN_LABELS = os.path.join(BASE_PATH, "labels")

# CV Parameters (will be tuned)
CV_CONFIG = {
    'roi_size': 50,           # ROI extraction radius
    'canny_low': 50,          # Canny lower threshold
    'canny_high': 150,        # Canny upper threshold
    'min_line_length': 20,    # Minimum error bar length
    'max_gap': 5,             # Max gap in line detection
    'vertical_tolerance': 3   # Max x deviation for vertical line
}

# Output directory
OUTPUT_DIR = "/kaggle/working"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Train images: {TRAIN_IMAGES}")
print(f"Train labels: {TRAIN_LABELS}")
print(f"Test images: {TEST_IMAGES}")
print(f"Test input labels: {TEST_INPUT_LABELS}")
print(f"Test ground truth: {TEST_GROUND_TRUTH}")
print(f"Output directory: {OUTPUT_DIR}")

Train images: /kaggle/input/graph-plots/Train/images
Train labels: /kaggle/input/graph-plots/Train/labels
Test images: /kaggle/input/graph-plots/Test/images
Test input labels: /kaggle/input/graph-plots/Test/test_labels
Test ground truth: /kaggle/input/graph-plots/Test/labels
Output directory: /kaggle/working


## 3. Data Loading Functions

In [4]:
def load_json_label(json_path: str) -> Dict:
    """Load label JSON file"""
    with open(json_path, 'r') as f:
        return json.load(f)

def load_training_data():
    """Load all training images and labels"""
    label_files = sorted([f for f in os.listdir(TRAIN_LABELS) if f.endswith('.json')])
    
    training_data = []
    
    print(f"Loading {len(label_files)} training samples...")
    
    for label_file in tqdm(label_files):
        label_data = load_json_label(os.path.join(TRAIN_LABELS, label_file))
        
        # Extract image filename from label
        if 'image_file' in label_data:
            image_file = label_data['image_file']
        else:
            # Assume same name with .png extension
            image_file = label_file.replace('.json', '.png')
        
        image_path = os.path.join(TRAIN_IMAGES, image_file)
        
        if not os.path.exists(image_path):
            continue
        
        # Parse data points
        for line_data in label_data:
            if 'points' not in line_data:
                continue
            
            line_name = line_data.get('label', {}).get('lineName', '')
            
            for point in line_data['points']:
                # Skip axis labels
                if point.get('label', '') in ['xmin', 'xmax', 'ymin', 'ymax']:
                    continue
                
                training_data.append({
                    'image_path': image_path,
                    'label_file': label_file,
                    'line_name': line_name,
                    'x': point['x'],
                    'y': point['y'],
                    'topBarPixelDistance': point.get('topBarPixelDistance', 0),
                    'bottomBarPixelDistance': point.get('bottomBarPixelDistance', 0)
                })
    
    df = pd.DataFrame(training_data)
    print(f"Loaded {len(df)} data points from {len(df['image_path'].unique())} images")
    
    return df

print("Data loading functions defined!")

Data loading functions defined!


## 4. CV Stem Detection Module

### Coarse Detection Strategy:
1. Extract ROI around data point
2. Apply edge detection (Canny)
3. Detect vertical lines (HoughLinesP)
4. Find line closest to point center
5. Extract coarse top/bottom distances

In [5]:
def extract_roi(image: np.ndarray, x: int, y: int, roi_size: int = 50) -> Tuple[np.ndarray, int, int]:
    """Extract ROI around data point"""
    h, w = image.shape[:2]
    
    x1 = max(0, x - roi_size)
    x2 = min(w, x + roi_size)
    y1 = max(0, y - roi_size)
    y2 = min(h, y + roi_size)
    
    roi = image[y1:y2, x1:x2]
    
    # Local coordinates of center point
    local_x = x - x1
    local_y = y - y1
    
    return roi, local_x, local_y

def detect_error_bar_cv(image_path: str, x: int, y: int, config: Dict) -> Dict:
    """
    CV-based coarse error bar detection.
    
    Returns:
        coarse_top_dist: estimated distance to top of error bar
        coarse_bottom_dist: estimated distance to bottom of error bar
        confidence: detection confidence [0-1]
    """
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return {'coarse_top_dist': 0, 'coarse_bottom_dist': 0, 'confidence': 0}
    
    # Extract ROI
    roi, local_x, local_y = extract_roi(img, int(x), int(y), config['roi_size'])
    
    if roi.size == 0:
        return {'coarse_top_dist': 0, 'coarse_bottom_dist': 0, 'confidence': 0}
    
    # Edge detection
    edges = cv2.Canny(roi, config['canny_low'], config['canny_high'])
    
    # Detect vertical lines
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi / 180,
        threshold=20,
        minLineLength=config['min_line_length'],
        maxLineGap=config['max_gap']
    )
    
    if lines is None:
        return {'coarse_top_dist': 0, 'coarse_bottom_dist': 0, 'confidence': 0}
    
    # Find vertical lines near the data point
    best_line = None
    min_dist = float('inf')
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        
        # Check if line is vertical
        if abs(x1 - x2) > config['vertical_tolerance']:
            continue
        
        # Check proximity to data point
        line_x = (x1 + x2) / 2
        dist = abs(line_x - local_x)
        
        if dist < min_dist:
            min_dist = dist
            best_line = (x1, y1, x2, y2)
    
    if best_line is None or min_dist > 10:
        return {'coarse_top_dist': 0, 'coarse_bottom_dist': 0, 'confidence': 0}
    
    # Calculate distances
    x1, y1, x2, y2 = best_line
    top_y = min(y1, y2)
    bottom_y = max(y1, y2)
    
    coarse_top_dist = abs(local_y - top_y)
    coarse_bottom_dist = abs(bottom_y - local_y)
    
    # Confidence based on proximity
    confidence = max(0, 1 - min_dist / 10)
    
    return {
        'coarse_top_dist': float(coarse_top_dist),
        'coarse_bottom_dist': float(coarse_bottom_dist),
        'confidence': float(confidence)
    }

print("CV stem detection module defined!")

CV stem detection module defined!


## 5. Feature Extraction for ML

### Features:
- ROI image features (HOG, intensity statistics)
- Coarse CV estimates (top_dist, bottom_dist)
- Confidence score
- Local context features

### 📊 Feature Engineering Details: 14 Refined Features

The hybrid pipeline uses **14 carefully selected features** to predict error bar distances:

#### **ROI Image Features (11 features)**

These features capture essential visual characteristics of the region around each data point:

**1-4. Basic Intensity Statistics**
- `mean_intensity`: Average pixel brightness in ROI
- `std_intensity`: Brightness variation (uniformity)
- `min_intensity`: Darkest pixel value
- `max_intensity`: Brightest pixel value

**5-6. Vertical Gradient Features**
- `grad_y_mean`: Average vertical edge strength (detects vertical lines)
- `grad_y_std`: Variation in vertical gradients

**7. Edge Density**
- `edge_density`: Percentage of edge pixels (Canny) in ROI
- Higher for error bars with clear boundaries

**8-11. Center Column Profile**
- `center_col_mean`: Average brightness of center vertical stripe
- `center_col_std`: Variation along center (detects stems)
- `center_col_min`: Darkest point in center
- `center_col_max`: Brightest point in center

#### **CV Coarse Detection Features (3 features)**

These are geometric estimates from computer vision:

**12. Coarse Top Distance**
- `cv_top_dist`: Estimated distance from data point to top of detected vertical line (pixels)
- From HoughLinesP line detection

**13. Coarse Bottom Distance**
- `cv_bottom_dist`: Estimated distance from data point to bottom of detected vertical line (pixels)
- From HoughLinesP line detection

**14. Confidence Score**
- `cv_confidence`: How confident CV is in detection (0-1)
- Based on proximity of detected line to data point
- Higher when vertical line is exactly at data point x-coordinate

### 🔄 How Features Are Created

```python
# Step 1: Extract ROI (100x100 window around point)
roi = image[y-50:y+50, x-50:x+50]

# Step 2: Calculate ROI features (11)
- Intensity stats from roi pixels
- Sobel gradients for vertical edges
- Canny edge detection for density
- Statistical analysis of center column

# Step 3: Run CV detection (3)
- Apply Canny edge detection
- HoughLinesP to find vertical lines
- Match nearest line to data point
- Extract top/bottom endpoints

# Step 4: Concatenate into feature vector
features = [roi_feat_1, ..., roi_feat_11, cv_top, cv_bottom, cv_conf]
# Total: 14 features
```

### 🎯 Why This Works

- **Refined feature set**: Removed redundant variance and padding features
- **ROI features** capture essential visual patterns (what error bars look like)
- **CV features** provide geometric structure (where error bars are)
- **ML model** learns to combine both for accurate predictions
- **Simpler = Better generalization**: 14 focused features vs 23 with redundancy

In [6]:
def extract_roi_features(image_path: str, x: int, y: int, roi_size: int = 50) -> np.ndarray:
    """
    Extract features from ROI for ML model.

    Features (11 total):
    - Basic intensity statistics (4)
    - Vertical gradient (2)
    - Edge density (1)
    - Center column profile (4)
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return np.zeros(11)

    roi, local_x, local_y = extract_roi(img, int(x), int(y), roi_size)

    if roi.size == 0:
        return np.zeros(11)

    features = []

    # Basic intensity statistics
    features.append(np.mean(roi))
    features.append(np.std(roi))
    features.append(np.min(roi))
    features.append(np.max(roi))

    # Vertical gradient
    grad_y = cv2.Sobel(roi, cv2.CV_64F, 0, 1, ksize=3)
    features.append(np.mean(np.abs(grad_y)))
    features.append(np.std(grad_y))

    # Edge density
    edges = cv2.Canny(roi, 50, 150)
    features.append(np.sum(edges > 0) / edges.size)

    # Center column profile
    if roi.shape[1] > 0:
        center_col = roi[:, roi.shape[1] // 2]
        features.append(np.mean(center_col))
        features.append(np.std(center_col))
        features.append(np.min(center_col))
        features.append(np.max(center_col))
    else:
        features.extend([0, 0, 0, 0])

    return np.array(features)


def create_ml_features(row: pd.Series, config: Dict) -> Tuple[np.ndarray, Dict]:
    """
    Create full feature vector for ML model.

    Features = [ROI features (11) + CV estimates (3)] = 14 features
    """
    # CV coarse estimates
    cv_result = detect_error_bar_cv(
        row['image_path'],
        row['x'],
        row['y'],
        config
    )

    # ROI features
    roi_features = extract_roi_features(
        row['image_path'],
        row['x'],
        row['y'],
        config['roi_size']
    )

    # Combine features
    features = np.concatenate([
        roi_features,
        [
            cv_result['coarse_top_dist'],
            cv_result['coarse_bottom_dist'],
            cv_result['confidence']
        ]
    ])

    return features, cv_result


print("Feature extraction functions defined!")

Feature extraction functions defined!


## 6. Load Training Data and Extract Features

In [7]:
# Load training data
train_df = load_training_data()

print(f"\nDataset statistics:")
print(f"Total points: {len(train_df)}")
print(f"Unique images: {train_df['image_path'].nunique()}")
print(f"\nError bar statistics:")
print(f"Mean top distance: {train_df['topBarPixelDistance'].mean():.2f}px")
print(f"Mean bottom distance: {train_df['bottomBarPixelDistance'].mean():.2f}px")
print(f"Points with error bars: {(train_df['topBarPixelDistance'] > 0).sum()} ({(train_df['topBarPixelDistance'] > 0).mean()*100:.1f}%)")

Loading 2400 training samples...


100%|██████████| 2400/2400 [00:24<00:00, 97.37it/s] 

Loaded 38087 data points from 2400 images

Dataset statistics:
Total points: 38087
Unique images: 2400

Error bar statistics:
Mean top distance: 44.46px
Mean bottom distance: 42.32px
Points with error bars: 32816 (86.2%)


In [8]:
# Extract features for all training points
print("Extracting features from training data...")

X_features = []
y_top = []
y_bottom = []
cv_estimates = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    features, cv_result = create_ml_features(row, CV_CONFIG)
    
    X_features.append(features)
    y_top.append(row['topBarPixelDistance'])
    y_bottom.append(row['bottomBarPixelDistance'])
    cv_estimates.append(cv_result)

X_train = np.array(X_features)
y_train_top = np.array(y_top)
y_train_bottom = np.array(y_bottom)

print(f"\nFeature matrix shape: {X_train.shape}")
print(f"Target top shape: {y_train_top.shape}")
print(f"Target bottom shape: {y_train_bottom.shape}")

# Store CV estimates in dataframe
train_df['cv_top_dist'] = [cv['coarse_top_dist'] for cv in cv_estimates]
train_df['cv_bottom_dist'] = [cv['coarse_bottom_dist'] for cv in cv_estimates]
train_df['cv_confidence'] = [cv['confidence'] for cv in cv_estimates]

Extracting features from training data...


100%|██████████| 38087/38087 [10:25<00:00, 60.93it/s]



Feature matrix shape: (38087, 14)
Target top shape: (38087,)
Target bottom shape: (38087,)


In [9]:
# Create comprehensive feature dataset for export
feature_names = [
    # ROI Features (1–11)
    'roi_mean_intensity', 'roi_std_intensity', 'roi_min_intensity', 'roi_max_intensity',
    'roi_grad_y_mean', 'roi_grad_y_std',
    'roi_edge_density',
    'roi_center_col_mean', 'roi_center_col_std', 'roi_center_col_min', 'roi_center_col_max',
    # CV Features (12–14)
    'cv_coarse_top_dist', 'cv_coarse_bottom_dist', 'cv_confidence'
]

# Create feature dataframe
feature_df = pd.DataFrame(X_train, columns=feature_names)

# Add metadata and targets
feature_df.insert(0, 'image_path', train_df['image_path'].values)
feature_df.insert(1, 'label_file', train_df['label_file'].values)
feature_df.insert(2, 'line_name', train_df['line_name'].values)
feature_df.insert(3, 'x', train_df['x'].values)
feature_df.insert(4, 'y', train_df['y'].values)
feature_df['target_top_dist'] = y_train_top
feature_df['target_bottom_dist'] = y_train_bottom

# Save to CSV
feature_csv_path = os.path.join(OUTPUT_DIR, 'hybrid_features_dataset.csv')
feature_df.to_csv(feature_csv_path, index=False)

print("=" * 80)
print("FEATURE DATASET EXPORTED")
print("=" * 80)
print(f"File: {feature_csv_path}")
print(f"Shape: {feature_df.shape}")
print(f"Columns: {len(feature_df.columns)}")
print("\nDataset Preview:")
print(feature_df.head(3).to_string())
print("\n" + "=" * 80)
print("\nFeature Statistics:")
print(feature_df[feature_names].describe().to_string())
print("=" * 80)

# Save feature descriptions
feature_descriptions = {
    'Feature': feature_names,
    'Category': (
        ['ROI Intensity'] * 4 +
        ['ROI Gradients'] * 2 +
        ['ROI Edges'] * 1 +
        ['ROI Center'] * 4 +
        ['CV Coarse'] * 3
    ),
    'Description': [
        'Mean pixel brightness in ROI',
        'Std deviation of pixel brightness',
        'Minimum pixel value',
        'Maximum pixel value',
        'Mean vertical gradient strength',
        'Std deviation of vertical gradients',
        'Percentage of edge pixels (Canny)',
        'Mean brightness of center column',
        'Std deviation of center column',
        'Min value in center column',
        'Max value in center column',
        'CV-detected top error bar distance',
        'CV-detected bottom error bar distance',
        'CV detection confidence (0–1)'
    ]
}

feature_desc_df = pd.DataFrame(feature_descriptions)
feature_desc_path = os.path.join(OUTPUT_DIR, 'feature_descriptions.csv')
feature_desc_df.to_csv(feature_desc_path, index=False)

print(f"\nFeature descriptions saved to: {feature_desc_path}")
print("\nYou can now examine the feature dataset before model training!")
print("This dataset shows exactly how CV features and ROI features combine.")


FEATURE DATASET EXPORTED
File: /kaggle/working/hybrid_features_dataset.csv
Shape: (38087, 21)
Columns: 21

Dataset Preview:
                                                                        image_path                                 label_file              line_name           x           y  roi_mean_intensity  roi_std_intensity  roi_min_intensity  roi_max_intensity  roi_grad_y_mean  roi_grad_y_std  roi_edge_density  roi_center_col_mean  roi_center_col_std  roi_center_col_min  roi_center_col_max  cv_coarse_top_dist  cv_coarse_bottom_dist  cv_confidence  target_top_dist  target_bottom_dist
0  /kaggle/input/graph-plots/Train/images/001f3fd0-23eb-4371-a52a-41b9b71c36bf.png  001f3fd0-23eb-4371-a52a-41b9b71c36bf.json  Daratumumab_500mg_BID  176.957721   94.156410            246.2709          30.608733               99.0              255.0          14.0922       66.247937            0.0816               254.64            2.242855               239.0               255.0                11

## 6a. Export Feature Dataset for Analysis

Before training, export the complete feature dataset for inspection and documentation.

## 7. Train Multiple Regression Models

### Models to Compare:
1. **Linear Regression** - Baseline
2. **Ridge Regression** - L2 regularization
3. **Random Forest** - Ensemble tree-based
4. **Gradient Boosting** - Sequential boosting
5. **MLP** - Small neural network

In [10]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Split for validation (80/20)
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr_top, y_val_top = train_test_split(
    X_train_scaled, y_train_top, test_size=0.2, random_state=42
)
_, _, y_tr_bottom, y_val_bottom = train_test_split(
    X_train_scaled, y_train_bottom, test_size=0.2, random_state=42
)

print(f"Training set: {X_tr.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")

Training set: 30469 samples
Validation set: 7618 samples


In [11]:
# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    'MLP': MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42, early_stopping=True)
}

# Train models for both top and bottom predictions
trained_models = {}
results = []

print("Training models...\n")

for name, model in models.items():
    print(f"Training {name}...")
    
    # Clone models for top and bottom
    from sklearn.base import clone
    model_top = clone(model)
    model_bottom = clone(model)
    
    # Train
    model_top.fit(X_tr, y_tr_top)
    model_bottom.fit(X_tr, y_tr_bottom)
    
    # Validate
    pred_top = model_top.predict(X_val)
    pred_bottom = model_bottom.predict(X_val)
    
    # Metrics
    mae_top = mean_absolute_error(y_val_top, pred_top)
    mae_bottom = mean_absolute_error(y_val_bottom, pred_bottom)
    rmse_top = np.sqrt(mean_squared_error(y_val_top, pred_top))
    rmse_bottom = np.sqrt(mean_squared_error(y_val_bottom, pred_bottom))
    
    # Store
    trained_models[name] = {'top': model_top, 'bottom': model_bottom}
    
    results.append({
        'Model': name,
        'MAE Top': mae_top,
        'MAE Bottom': mae_bottom,
        'MAE Avg': (mae_top + mae_bottom) / 2,
        'RMSE Top': rmse_top,
        'RMSE Bottom': rmse_bottom,
        'RMSE Avg': (rmse_top + rmse_bottom) / 2
    })
    
    print(f"  MAE Top: {mae_top:.2f}px, MAE Bottom: {mae_bottom:.2f}px")
    print(f"  RMSE Top: {rmse_top:.2f}px, RMSE Bottom: {rmse_bottom:.2f}px\n")

# Results table
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("VALIDATION RESULTS")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

Training models...

Training Linear Regression...
  MAE Top: 32.51px, MAE Bottom: 29.89px
  RMSE Top: 219.91px, RMSE Bottom: 223.70px

Training Ridge Regression...
  MAE Top: 32.51px, MAE Bottom: 29.89px
  RMSE Top: 219.91px, RMSE Bottom: 223.70px

Training Random Forest...
  MAE Top: 33.83px, MAE Bottom: 32.46px
  RMSE Top: 282.28px, RMSE Bottom: 239.53px

Training Gradient Boosting...
  MAE Top: 35.71px, MAE Bottom: 31.83px
  RMSE Top: 410.13px, RMSE Bottom: 213.29px

Training MLP...
  MAE Top: 31.49px, MAE Bottom: 28.77px
  RMSE Top: 220.14px, RMSE Bottom: 223.61px


VALIDATION RESULTS
            Model   MAE Top  MAE Bottom   MAE Avg   RMSE Top  RMSE Bottom   RMSE Avg
Linear Regression 32.511818   29.894091 31.202954 219.906666   223.704247 221.805457
 Ridge Regression 32.511230   29.893631 31.202431 219.906582   223.704136 221.805359
    Random Forest 33.832784   32.462999 33.147892 282.280307   239.526310 260.903308
Gradient Boosting 35.711747   31.826737 33.769242 410.130937   2

## 8. Hyperparameter Tuning

Tune the best performing model from above.

In [12]:
# Select best model based on validation results
best_model_name = results_df.loc[results_df['MAE Avg'].idxmin(), 'Model']
print(f"Best model: {best_model_name}")
print(f"Performing hyperparameter tuning...\n")

# Hyperparameter grids
param_grids = {
    'Ridge Regression': {
        'alpha': [0.1, 1.0, 10.0, 100.0]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5, 10]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'MLP': {
        'hidden_layer_sizes': [(32,), (64,), (64, 32), (128, 64)],
        'alpha': [0.0001, 0.001, 0.01]
    }
}

if best_model_name in param_grids:
    # Grid search for top prediction
    base_model = models[best_model_name]
    grid_search = GridSearchCV(
        base_model,
        param_grids[best_model_name],
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=1
    )
    
    print("Tuning for top error bar prediction...")
    grid_search.fit(X_tr, y_tr_top)
    best_model_top = grid_search.best_estimator_
    
    print(f"Best params (top): {grid_search.best_params_}")
    print(f"Best CV score (top): {-grid_search.best_score_:.2f}px MAE\n")
    
    # Grid search for bottom prediction
    grid_search = GridSearchCV(
        base_model,
        param_grids[best_model_name],
        cv=3,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        verbose=1
    )
    
    print("Tuning for bottom error bar prediction...")
    grid_search.fit(X_tr, y_tr_bottom)
    best_model_bottom = grid_search.best_estimator_
    
    print(f"Best params (bottom): {grid_search.best_params_}")
    print(f"Best CV score (bottom): {-grid_search.best_score_:.2f}px MAE")
    
    # Update trained models
    trained_models['Tuned ' + best_model_name] = {
        'top': best_model_top,
        'bottom': best_model_bottom
    }
else:
    print(f"No tuning grid defined for {best_model_name}")

Best model: MLP
Performing hyperparameter tuning...

Tuning for top error bar prediction...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params (top): {'alpha': 0.0001, 'hidden_layer_sizes': (64, 32)}
Best CV score (top): 31.99px MAE

Tuning for bottom error bar prediction...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params (bottom): {'alpha': 0.01, 'hidden_layer_sizes': (64,)}
Best CV score (bottom): 28.69px MAE


## 9. Ablation Study

Compare:
- **CV Only** - Pure computer vision
- **ML Only** - No CV features
- **Hybrid** - CV + ML (full pipeline)

In [13]:
print("Running ablation study...\n")

ablation_results = []

# 1. CV ONLY
# Use CV estimates from validation split
val_indices = train_test_split(
    range(len(train_df)), test_size=0.2, random_state=42
)[1]

cv_pred_top = train_df.iloc[val_indices]['cv_top_dist'].values
cv_pred_bottom = train_df.iloc[val_indices]['cv_bottom_dist'].values

mae_cv_top = mean_absolute_error(y_val_top, cv_pred_top)
mae_cv_bottom = mean_absolute_error(y_val_bottom, cv_pred_bottom)
rmse_cv_top = np.sqrt(mean_squared_error(y_val_top, cv_pred_top))
rmse_cv_bottom = np.sqrt(mean_squared_error(y_val_bottom, cv_pred_bottom))

ablation_results.append({
    'Method': 'CV Only',
    'MAE Top': mae_cv_top,
    'MAE Bottom': mae_cv_bottom,
    'MAE Avg': (mae_cv_top + mae_cv_bottom) / 2,
    'RMSE Avg': (rmse_cv_top + rmse_cv_bottom) / 2
})

print(f"CV Only - MAE Top: {mae_cv_top:.2f}px, MAE Bottom: {mae_cv_bottom:.2f}px\n")

# 2. ML ONLY (no CV features)
# Use only first 11 features (ROI features, no CV estimates)
X_tr_ml_only = X_tr[:, :11]
X_val_ml_only = X_val[:, :11]

ml_only = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
ml_only_top = clone(ml_only)
ml_only_bottom = clone(ml_only)

ml_only_top.fit(X_tr_ml_only, y_tr_top)
ml_only_bottom.fit(X_tr_ml_only, y_tr_bottom)

ml_pred_top = ml_only_top.predict(X_val_ml_only)
ml_pred_bottom = ml_only_bottom.predict(X_val_ml_only)

mae_ml_top = mean_absolute_error(y_val_top, ml_pred_top)
mae_ml_bottom = mean_absolute_error(y_val_bottom, ml_pred_bottom)
rmse_ml_top = np.sqrt(mean_squared_error(y_val_top, ml_pred_top))
rmse_ml_bottom = np.sqrt(mean_squared_error(y_val_bottom, ml_pred_bottom))

ablation_results.append({
    'Method': 'ML Only (no CV features)',
    'MAE Top': mae_ml_top,
    'MAE Bottom': mae_ml_bottom,
    'MAE Avg': (mae_ml_top + mae_ml_bottom) / 2,
    'RMSE Avg': (rmse_ml_top + rmse_ml_bottom) / 2
})

print(f"ML Only - MAE Top: {mae_ml_top:.2f}px, MAE Bottom: {mae_ml_bottom:.2f}px\n")

# 3. HYBRID (full pipeline)
best_hybrid = trained_models[best_model_name]
hybrid_pred_top = best_hybrid['top'].predict(X_val)
hybrid_pred_bottom = best_hybrid['bottom'].predict(X_val)

mae_hybrid_top = mean_absolute_error(y_val_top, hybrid_pred_top)
mae_hybrid_bottom = mean_absolute_error(y_val_bottom, hybrid_pred_bottom)
rmse_hybrid_top = np.sqrt(mean_squared_error(y_val_top, hybrid_pred_top))
rmse_hybrid_bottom = np.sqrt(mean_squared_error(y_val_bottom, hybrid_pred_bottom))

ablation_results.append({
    'Method': f'Hybrid (CV + {best_model_name})',
    'MAE Top': mae_hybrid_top,
    'MAE Bottom': mae_hybrid_bottom,
    'MAE Avg': (mae_hybrid_top + mae_hybrid_bottom) / 2,
    'RMSE Avg': (rmse_hybrid_top + rmse_hybrid_bottom) / 2
})

print(f"Hybrid - MAE Top: {mae_hybrid_top:.2f}px, MAE Bottom: {mae_hybrid_bottom:.2f}px\n")

# Display ablation results
ablation_df = pd.DataFrame(ablation_results)
print("\n" + "="*80)
print("ABLATION STUDY RESULTS")
print("="*80)
print(ablation_df.to_string(index=False))
print("="*80)

# Save ablation results
ablation_df.to_csv(os.path.join(OUTPUT_DIR, 'ablation_study.csv'), index=False)
print(f"\nAblation results saved to {OUTPUT_DIR}/ablation_study.csv")

Running ablation study...

CV Only - MAE Top: 29.34px, MAE Bottom: 27.12px

ML Only - MAE Top: 32.66px, MAE Bottom: 32.06px

Hybrid - MAE Top: 31.49px, MAE Bottom: 28.77px


ABLATION STUDY RESULTS
                  Method   MAE Top  MAE Bottom   MAE Avg   RMSE Avg
                 CV Only 29.341308   27.120453 28.230880 223.499885
ML Only (no CV features) 32.663262   32.062029 32.362646 183.587597
       Hybrid (CV + MLP) 31.489044   28.770441 30.129742 221.873797

Ablation results saved to /kaggle/working/ablation_study.csv


## 10. Select Final Model and Retrain on Full Training Data

In [14]:
# Use best hybrid model
print(f"Final model: Hybrid (CV + {best_model_name})\n")

# Retrain on full training data
print("Retraining on full training set...")

final_model_top = clone(trained_models[best_model_name]['top'])
final_model_bottom = clone(trained_models[best_model_name]['bottom'])

final_model_top.fit(X_train_scaled, y_train_top)
final_model_bottom.fit(X_train_scaled, y_train_bottom)

print("Final models trained on full training set!")

Final model: Hybrid (CV + MLP)

Retraining on full training set...
Final models trained on full training set!


## 11. Evaluation Functions

In [15]:
def calculate_metrics(y_true_top, y_true_bottom, y_pred_top, y_pred_bottom):
    """Calculate comprehensive evaluation metrics"""
    
    # Absolute errors
    errors_top = np.abs(y_true_top - y_pred_top)
    errors_bottom = np.abs(y_true_bottom - y_pred_bottom)
    errors_mean = (errors_top + errors_bottom) / 2
    
    # Accuracy at thresholds
    acc_5px = np.mean(errors_mean <= 5) * 100
    acc_10px = np.mean(errors_mean <= 10) * 100
    acc_20px = np.mean(errors_mean <= 20) * 100
    
    metrics = {
        'num_points': len(y_true_top),
        'mean_top_error': np.mean(errors_top),
        'mean_bottom_error': np.mean(errors_bottom),
        'mean_overall_error': np.mean(errors_mean),
        'median_top_error': np.median(errors_top),
        'median_bottom_error': np.median(errors_bottom),
        'median_overall_error': np.median(errors_mean),
        'rmse_top': np.sqrt(mean_squared_error(y_true_top, y_pred_top)),
        'rmse_bottom': np.sqrt(mean_squared_error(y_true_bottom, y_pred_bottom)),
        'rmse_overall': np.sqrt(mean_squared_error(
            np.concatenate([y_true_top, y_true_bottom]),
            np.concatenate([y_pred_top, y_pred_bottom])
        )),
        'accuracy_5px': acc_5px,
        'accuracy_10px': acc_10px,
        'accuracy_20px': acc_20px,
        'std_top_error': np.std(errors_top),
        'std_bottom_error': np.std(errors_bottom),
        'max_top_error': np.max(errors_top),
        'max_bottom_error': np.max(errors_bottom)
    }
    
    return metrics

def print_metrics(metrics, title="METRICS"):
    """Pretty print metrics"""
    print("\n" + "="*80)
    print(title)
    print("="*80)
    print(f"Total Points: {metrics['num_points']}")
    print(f"\nMean Errors:")
    print(f"  Top Error:     {metrics['mean_top_error']:.2f}px")
    print(f"  Bottom Error:  {metrics['mean_bottom_error']:.2f}px")
    print(f"  Overall Error: {metrics['mean_overall_error']:.2f}px")
    print(f"\nMedian Errors:")
    print(f"  Top Error:     {metrics['median_top_error']:.2f}px")
    print(f"  Bottom Error:  {metrics['median_bottom_error']:.2f}px")
    print(f"  Overall Error: {metrics['median_overall_error']:.2f}px")
    print(f"\nRMSE:")
    print(f"  Top:     {metrics['rmse_top']:.2f}px")
    print(f"  Bottom:  {metrics['rmse_bottom']:.2f}px")
    print(f"  Overall: {metrics['rmse_overall']:.2f}px")
    print(f"\nAccuracy @ Threshold:")
    print(f"  Within 5px:  {metrics['accuracy_5px']:.1f}%")
    print(f"  Within 10px: {metrics['accuracy_10px']:.1f}%")
    print(f"  Within 20px: {metrics['accuracy_20px']:.1f}%")
    print("="*80)

print("Evaluation functions defined!")

Evaluation functions defined!


## 12. Validation Set Evaluation

In [16]:
# Evaluate on validation set
val_pred_top = final_model_top.predict(X_val)
val_pred_bottom = final_model_bottom.predict(X_val)

val_metrics = calculate_metrics(y_val_top, y_val_bottom, val_pred_top, val_pred_bottom)
print_metrics(val_metrics, "VALIDATION SET RESULTS (Hybrid CV+ML)")


VALIDATION SET RESULTS (Hybrid CV+ML)
Total Points: 7618

Mean Errors:
  Top Error:     30.58px
  Bottom Error:  29.86px
  Overall Error: 30.22px

Median Errors:
  Top Error:     15.06px
  Bottom Error:  14.67px
  Overall Error: 16.14px

RMSE:
  Top:     217.75px
  Bottom:  216.64px
  Overall: 217.20px

Accuracy @ Threshold:
  Within 5px:  12.2%
  Within 10px: 31.5%
  Within 20px: 59.1%


## 13. Save Models and Results

In [17]:
import pickle

# Save models
models_to_save = {
    'model_top': final_model_top,
    'model_bottom': final_model_bottom,
    'scaler': scaler,
    'config': CV_CONFIG,
    'best_model_name': best_model_name
}

with open(os.path.join(OUTPUT_DIR, 'hybrid_models.pkl'), 'wb') as f:
    pickle.dump(models_to_save, f)

print(f"Models saved to {OUTPUT_DIR}/hybrid_models.pkl")

# Save validation metrics
val_metrics_df = pd.DataFrame([val_metrics])
val_metrics_df.to_csv(os.path.join(OUTPUT_DIR, 'validation_metrics.csv'), index=False)

print(f"Validation metrics saved to {OUTPUT_DIR}/validation_metrics.csv")

Models saved to /kaggle/working/hybrid_models.pkl
Validation metrics saved to /kaggle/working/validation_metrics.csv


## 14. Test Set Prediction Function

Define the prediction function that will be used for test set evaluation.

In [18]:
def predict_error_bars(image_path: str, data_points: List[Dict], model_top, model_bottom, scaler, config: Dict) -> List[Dict]:
    """
    Predict error bars for a list of data points in an image.
    """
    predictions = []
    
    for point in data_points:
        x = point['x']
        y = point['y']
        
        # Extract features
        cv_result = detect_error_bar_cv(image_path, x, y, config)
        roi_features = extract_roi_features(image_path, x, y, config['roi_size'])
        
        # Combine features
        features = np.concatenate([
            roi_features,
            [cv_result['coarse_top_dist'],
             cv_result['coarse_bottom_dist'],
             cv_result['confidence']]
        ])
        
        # Scale and predict
        features_scaled = scaler.transform(features.reshape(1, -1))
        pred_top = model_top.predict(features_scaled)[0]
        pred_bottom = model_bottom.predict(features_scaled)[0]
        
        # Ensure non-negative
        pred_top = max(0, pred_top)
        pred_bottom = max(0, pred_bottom)
        
        predictions.append({
            'x': x,
            'y': y,
            'topBarPixelDistance': float(pred_top),
            'bottomBarPixelDistance': float(pred_bottom),
            'deviationPixelDistance': float(max(pred_top, pred_bottom)),
            'label': point.get('label', '')
        })
    
    return predictions

print("Test prediction function defined!")

Test prediction function defined!


## 15. Process Test Set (600 samples)

Evaluate the best trained model on the test set.

In [20]:
# Process test set with 600-sample limit
print("Processing test set (600 samples)...\n")

# Initialize all required variables
all_test_predictions = {}
test_detailed_results = []
all_model_predictions = {}
all_model_test_metrics = {}

test_files = sorted([f for f in os.listdir(TEST_INPUT_LABELS) if f.endswith('.json')])

# Limit to 600 samples for evaluation
import random
random.seed(42)
if len(test_files) > 600:
    test_files = random.sample(test_files, 600)
    print(f"Randomly selected 600 files from {len(os.listdir(TEST_INPUT_LABELS))} total test files")
else:
    print(f"Using all {len(test_files)} test files (less than 600)")

test_processed = 0
test_failed = 0

for test_file in tqdm(test_files):
    try:
        # Load input
        input_path = os.path.join(TEST_INPUT_LABELS, test_file)
        with open(input_path, 'r') as f:
            input_data = json.load(f)
        
        image_file = input_data.get('image_file', test_file.replace('.json', '.png'))
        image_path = os.path.join(TEST_IMAGES, image_file)
        
        if not os.path.exists(image_path):
            test_failed += 1
            continue
        
        # Predict for each line
        predictions = []
        
        for line_data in input_data.get('data_points', []):
            line_name = line_data.get('lineName', '')
            points = line_data.get('points', [])
            
            # Predict error bars
            pred_points = predict_error_bars(
                image_path, points, 
                final_model_top, final_model_bottom, 
                scaler, CV_CONFIG
            )
            
            predictions.append({
                'label': {'lineName': line_name},
                'points': pred_points
            })
        
        # Store predictions
        all_test_predictions[test_file] = {
            'image_file': image_file,
            'predictions': predictions
        }
        
        # Evaluate against ground truth
        gt_path = os.path.join(TEST_GROUND_TRUTH, test_file)
        if os.path.exists(gt_path):
            with open(gt_path, 'r') as f:
                gt_data = json.load(f)
            
            # Calculate errors for this image
            image_errors_top = []
            image_errors_bottom = []
            
            for pred_line in predictions:
                # Find matching ground truth line
                gt_line = None
                for gt in gt_data:
                    if gt.get('label', {}).get('lineName') == pred_line.get('label', {}).get('lineName'):
                        gt_line = gt
                        break
                
                if gt_line is None:
                    continue
                
                pred_points = [p for p in pred_line['points'] if p.get('label', '') not in ['xmin', 'xmax', 'ymin', 'ymax']]
                gt_points = [p for p in gt_line['points'] if p.get('label', '') not in ['xmin', 'xmax', 'ymin', 'ymax']]
                
                for pred_pt, gt_pt in zip(pred_points, gt_points):
                    image_errors_top.append(abs(pred_pt['topBarPixelDistance'] - gt_pt.get('topBarPixelDistance', 0)))
                    image_errors_bottom.append(abs(pred_pt['bottomBarPixelDistance'] - gt_pt.get('bottomBarPixelDistance', 0)))
            
            if image_errors_top:
                test_detailed_results.append({
                    'image_file': image_file,
                    'num_points': len(image_errors_top),
                    'mean_top_error': np.mean(image_errors_top),
                    'mean_bottom_error': np.mean(image_errors_bottom),
                    'mean_overall_error': np.mean(image_errors_top + image_errors_bottom)
                })
        
        test_processed += 1
        
    except Exception as e:
        test_failed += 1
        if test_failed <= 5:
            print(f"Error processing {test_file}: {e}")

print(f"\n{'='*80}")
print(f"TEST SET PROCESSING COMPLETE (600 SAMPLES)")
print(f"{'='*80}")
print(f"Total selected: {len(test_files)} files")
print(f"Processed: {test_processed} files")
print(f"Failed: {test_failed} files")
print(f"{'='*80}")

Processing test set (600 samples)...

Using all 600 test files (less than 600)


100%|██████████| 600/600 [02:59<00:00,  3.34it/s]


TEST SET PROCESSING COMPLETE (600 SAMPLES)
Total selected: 600 files
Processed: 600 files
Failed: 0 files


In [21]:
# Generate predictions for ALL models on the same 600-sample test set
print("Generating predictions for all models on 600-sample test set...\n")

# Add CV-only baseline
trained_models['CV Only (Baseline)'] = None  # Special marker for CV-only

for model_name in tqdm(trained_models.keys(), desc="Processing models"):
    print(f"\n{'='*80}")
    print(f"MODEL: {model_name}")
    print(f"{'='*80}")
    
    model_predictions = {}
    model_test_results = []
    test_processed = 0
    test_failed = 0
    
    # Use the same test_files subset (600 samples)
    for test_file in test_files:
        try:
            # Load input
            input_path = os.path.join(TEST_INPUT_LABELS, test_file)
            with open(input_path, 'r') as f:
                input_data = json.load(f)
            
            image_file = input_data.get('image_file', test_file.replace('.json', '.png'))
            image_path = os.path.join(TEST_IMAGES, image_file)
            
            if not os.path.exists(image_path):
                test_failed += 1
                continue
            
            # Predict for each line
            predictions = []
            
            for line_data in input_data.get('data_points', []):
                line_name = line_data.get('lineName', '')
                points = line_data.get('points', [])
                
                # Predict based on model type
                if model_name == 'CV Only (Baseline)':
                    # Use CV estimates directly
                    pred_points = []
                    for point in points:
                        cv_result = detect_error_bar_cv(image_path, point['x'], point['y'], CV_CONFIG)
                        pred_points.append({
                            'x': point['x'],
                            'y': point['y'],
                            'topBarPixelDistance': float(cv_result['coarse_top_dist']),
                            'bottomBarPixelDistance': float(cv_result['coarse_bottom_dist']),
                            'deviationPixelDistance': float(max(cv_result['coarse_top_dist'], cv_result['coarse_bottom_dist'])),
                            'label': point.get('label', '')
                        })
                else:
                    # Use ML model
                    model_top = trained_models[model_name]['top']
                    model_bottom = trained_models[model_name]['bottom']
                    pred_points = predict_error_bars(
                        image_path, points, 
                        model_top, model_bottom, 
                        scaler, CV_CONFIG
                    )
                
                predictions.append({
                    'label': {'lineName': line_name},
                    'points': pred_points
                })
            
            # Store predictions
            model_predictions[test_file] = {
                'image_file': image_file,
                'predictions': predictions
            }
            
            # Evaluate against ground truth
            gt_path = os.path.join(TEST_GROUND_TRUTH, test_file)
            if os.path.exists(gt_path):
                with open(gt_path, 'r') as f:
                    gt_data = json.load(f)
                
                image_errors_top = []
                image_errors_bottom = []
                
                for pred_line in predictions:
                    gt_line = None
                    for gt in gt_data:
                        if gt.get('label', {}).get('lineName') == pred_line.get('label', {}).get('lineName'):
                            gt_line = gt
                            break
                    
                    if gt_line is None:
                        continue
                    
                    pred_points = [p for p in pred_line['points'] if p.get('label', '') not in ['xmin', 'xmax', 'ymin', 'ymax']]
                    gt_points = [p for p in gt_line['points'] if p.get('label', '') not in ['xmin', 'xmax', 'ymin', 'ymax']]
                    
                    for pred_pt, gt_pt in zip(pred_points, gt_points):
                        image_errors_top.append(abs(pred_pt['topBarPixelDistance'] - gt_pt.get('topBarPixelDistance', 0)))
                        image_errors_bottom.append(abs(pred_pt['bottomBarPixelDistance'] - gt_pt.get('bottomBarPixelDistance', 0)))
                
                if image_errors_top:
                    model_test_results.append({
                        'image_file': image_file,
                        'num_points': len(image_errors_top),
                        'mean_top_error': np.mean(image_errors_top),
                        'mean_bottom_error': np.mean(image_errors_bottom),
                        'mean_overall_error': np.mean(image_errors_top + image_errors_bottom)
                    })
            
            test_processed += 1
            
        except Exception as e:
            test_failed += 1
            if test_failed <= 3:
                print(f"Error: {test_file}: {e}")
    
    # Calculate model metrics
    if model_test_results:
        all_test_top_errors = []
        all_test_bottom_errors = []
        
        for result in model_test_results:
            all_test_top_errors.extend([result['mean_top_error']] * result['num_points'])
            all_test_bottom_errors.extend([result['mean_bottom_error']] * result['num_points'])
        
        all_test_mean_errors = [(t + b) / 2 for t, b in zip(all_test_top_errors, all_test_bottom_errors)]
        
        model_metrics = {
            'model_name': model_name,
            'num_images': len(model_test_results),
            'num_points': sum(r['num_points'] for r in model_test_results),
            'mean_top_error': np.mean(all_test_top_errors),
            'mean_bottom_error': np.mean(all_test_bottom_errors),
            'mean_overall_error': np.mean(all_test_mean_errors),
            'median_top_error': np.median(all_test_top_errors),
            'median_bottom_error': np.median(all_test_bottom_errors),
            'median_overall_error': np.median(all_test_mean_errors),
            'rmse_top': np.sqrt(np.mean(np.array(all_test_top_errors)**2)),
            'rmse_bottom': np.sqrt(np.mean(np.array(all_test_bottom_errors)**2)),
            'rmse_overall': np.sqrt(np.mean(np.array(all_test_mean_errors)**2)),
            'accuracy_5px': np.mean(np.array(all_test_mean_errors) <= 5) * 100,
            'accuracy_10px': np.mean(np.array(all_test_mean_errors) <= 10) * 100,
            'accuracy_20px': np.mean(np.array(all_test_mean_errors) <= 20) * 100,
            'std_top_error': np.std(all_test_top_errors),
            'std_bottom_error': np.std(all_test_bottom_errors)
        }
        
        all_model_test_metrics[model_name] = model_metrics
        all_model_predictions[model_name] = model_predictions
        
        print(f"\nProcessed: {test_processed} images")
        print(f"Mean Error: {model_metrics['mean_overall_error']:.2f}px")
        print(f"Accuracy@10px: {model_metrics['accuracy_10px']:.1f}%")


print(f"\n{'='*80}")
print(f"ALL MODELS PROCESSED ON 600-SAMPLE TEST SET")
print(f"{'='*80}")

Generating predictions for all models on 600-sample test set...



Processing models:   0%|          | 0/7 [00:00<?, ?it/s]


MODEL: Linear Regression


Processing models:  14%|█▍        | 1/7 [02:47<16:46, 167.83s/it]


Processed: 600 images
Mean Error: 24.79px
Accuracy@10px: 1.0%

MODEL: Ridge Regression


...


Processed: 600 images
Mean Error: 23.93px
Accuracy@10px: 2.7%

MODEL: Tuned MLP


Processing models:  86%|████████▌ | 6/7 [25:08<03:48, 228.40s/it]


Processed: 600 images
Mean Error: 24.04px
Accuracy@10px: 2.3%

MODEL: CV Only (Baseline)


Processing models: 100%|██████████| 7/7 [26:30<00:00, 227.18s/it]


Processed: 600 images
Mean Error: 21.07px
Accuracy@10px: 10.5%

ALL MODELS PROCESSED ON 600-SAMPLE TEST SET


In [22]:
# FINAL COMPREHENSIVE COMPARISON TABLE (600 Test Samples)
print("\n" + "="*120)
print("FINAL MODEL COMPARISON - ALL APPROACHES (600 TEST SAMPLES)")
print("="*120)

comparison_data = []

for model_name, metrics in all_model_test_metrics.items():
    comparison_data.append({
        'Approach': model_name,
        'Images': metrics['num_images'],
        'Points': metrics['num_points'],
        'Mean Error (px)': round(metrics['mean_overall_error'], 2),
        'Median Error (px)': round(metrics['median_overall_error'], 2),
        'RMSE (px)': round(metrics['rmse_overall'], 2),
        'Std Dev (px)': round((metrics['std_top_error'] + metrics['std_bottom_error']) / 2, 2),
        'Acc@5px (%)': round(metrics['accuracy_5px'], 1),
        'Acc@10px (%)': round(metrics['accuracy_10px'], 1),
        'Acc@20px (%)': round(metrics['accuracy_20px'], 1)
    })

comparison_df = pd.DataFrame(comparison_data)

# Sort by mean error (best first)
comparison_df = comparison_df.sort_values('Mean Error (px)')

# Add rank column
comparison_df.insert(0, 'Rank', range(1, len(comparison_df) + 1))

print("\n" + comparison_df.to_string(index=False))
print("\n" + "="*120)

# Save comprehensive comparison table
comparison_csv = os.path.join(OUTPUT_DIR, 'final_model_comparison_600samples.csv')
comparison_df.to_csv(comparison_csv, index=False)
print(f"\nFinal comparison table saved to: {comparison_csv}")

# Identify best model
best_model_name = comparison_df.iloc[0]['Approach']
best_model_error = comparison_df.iloc[0]['Mean Error (px)']

print(f"\n{'='*120}")
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"Mean Overall Error: {best_model_error:.2f}px")
print(f"{'='*120}")

# Performance improvement analysis
if 'CV Only (Baseline)' in all_model_test_metrics:
    baseline_error = all_model_test_metrics['CV Only (Baseline)']['mean_overall_error']
    
    print(f"\n{'='*120}")
    print("IMPROVEMENT OVER CV BASELINE")
    print("="*120)
    
    improvement_data = []
    for model_name, metrics in all_model_test_metrics.items():
        if model_name != 'CV Only (Baseline)':
            improvement_pct = ((baseline_error - metrics['mean_overall_error']) / baseline_error) * 100
            improvement_px = baseline_error - metrics['mean_overall_error']
            improvement_data.append({
                'Approach': model_name,
                'Mean Error (px)': round(metrics['mean_overall_error'], 2),
                'Baseline Error (px)': round(baseline_error, 2),
                'Improvement (%)': round(improvement_pct, 1),
                'Error Reduction (px)': round(improvement_px, 2)
            })
    
    improvement_df = pd.DataFrame(improvement_data)
    improvement_df = improvement_df.sort_values('Improvement (%)', ascending=False)
    print("\n" + improvement_df.to_string(index=False))
    print("="*120)
    
    # Save improvement table
    improvement_csv = os.path.join(OUTPUT_DIR, 'improvement_over_baseline_600samples.csv')
    improvement_df.to_csv(improvement_csv, index=False)
    print(f"\nImprovement analysis saved to: {improvement_csv}")

# Summary statistics
print(f"\n{'='*120}")
print("SUMMARY STATISTICS")
print("="*120)
print(f"Test Set Size: 600 samples")
print(f"Total Models Evaluated: {len(comparison_df)}")
print(f"\nBest Performance:")
print(f"  Model: {comparison_df.iloc[0]['Approach']}")
print(f"  Mean Error: {comparison_df.iloc[0]['Mean Error (px)']:.2f}px")
print(f"  Accuracy@10px: {comparison_df.iloc[0]['Acc@10px (%)']:.1f}%")
print(f"\nWorst Performance:")
print(f"  Model: {comparison_df.iloc[-1]['Approach']}")
print(f"  Mean Error: {comparison_df.iloc[-1]['Mean Error (px)']:.2f}px")
print(f"  Accuracy@10px: {comparison_df.iloc[-1]['Acc@10px (%)']:.1f}%")
print(f"\nAverage Across All Models:")
print(f"  Mean Error: {comparison_df['Mean Error (px)'].mean():.2f}px")
print(f"  RMSE: {comparison_df['RMSE (px)'].mean():.2f}px")
print(f"  Accuracy@10px: {comparison_df['Acc@10px (%)'].mean():.1f}%")
print("="*120)

print("\n✅ FINAL COMPARISON COMPLETE - ALL APPROACHES EVALUATED ON 600 TEST SAMPLES")
print("✅ Results saved to CSV files for documentation")


FINAL MODEL COMPARISON - ALL APPROACHES (600 TEST SAMPLES)

 Rank           Approach  Images  Points  Mean Error (px)  Median Error (px)  RMSE (px)  Std Dev (px)  Acc@5px (%)  Acc@10px (%)  Acc@20px (%)
    1 CV Only (Baseline)     600   10229            21.07              15.54      37.74         31.84          0.5          10.5          72.7
    2                MLP     600   10229            23.93              19.68      36.36         27.89          0.0           2.7          52.3
    3          Tuned MLP     600   10229            24.04              19.78      36.26         27.79          0.1           2.3          51.0
    4  Linear Regression     600   10229            24.79              20.65      36.70         27.53          0.1           1.0          44.2
    5   Ridge Regression     600   10229            24.79              20.65      36.70         27.53          0.1           1.0          44.2
    6  Gradient Boosting     600   10229            27.67              16.40     

## 17. Final Model Comparison Table

Compare all approaches and identify the best performing model.

In [23]:
from datetime import datetime
import zipfile

# Export BEST model predictions only
print(f"\nExporting predictions for BEST model: {best_model_name}\n")

# Get predictions for best model
best_predictions = all_model_predictions[best_model_name]

# Create predictions directory
pred_dir = os.path.join(OUTPUT_DIR, "predictions")
os.makedirs(pred_dir, exist_ok=True)

# Save JSON files
saved_count = 0
for test_file, pred_data in best_predictions.items():
    output_data = {
        'image_file': pred_data['image_file'],
        'model': best_model_name,
        'data_points': pred_data['predictions']
    }
    
    output_path = os.path.join(pred_dir, test_file)
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=2)
    
    saved_count += 1

# Create ZIP file
zip_filename = os.path.join(OUTPUT_DIR, f"best_model_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add predictions
    for test_file in os.listdir(pred_dir):
        if test_file.endswith('.json'):
            file_path = os.path.join(pred_dir, test_file)
            zipf.write(file_path, arcname=f"predictions/{test_file}")
    
    # Add model comparison table
    if os.path.exists(comparison_csv):
        zipf.write(comparison_csv, arcname='model_comparison_table.csv')
    
    # Add feature dataset
    feature_csv = os.path.join(OUTPUT_DIR, 'hybrid_features_dataset.csv')
    if os.path.exists(feature_csv):
        zipf.write(feature_csv, arcname='hybrid_features_dataset.csv')
    
    # Add feature descriptions
    feature_desc = os.path.join(OUTPUT_DIR, 'feature_descriptions.csv')
    if os.path.exists(feature_desc):
        zipf.write(feature_desc, arcname='feature_descriptions.csv')
    
    # Add validation metrics
    val_metrics_file = os.path.join(OUTPUT_DIR, 'validation_metrics.csv')
    if os.path.exists(val_metrics_file):
        zipf.write(val_metrics_file, arcname='validation_metrics.csv')
    
    # Add ablation study
    ablation_file = os.path.join(OUTPUT_DIR, 'ablation_study.csv')
    if os.path.exists(ablation_file):
        zipf.write(ablation_file, arcname='ablation_study.csv')

zip_size_mb = os.path.getsize(zip_filename) / (1024 * 1024)

print("="*80)
print("EXPORT COMPLETE")
print("="*80)
print(f"Best Model: {best_model_name}")
print(f"Predictions: {saved_count} files")
print(f"ZIP File: {zip_filename}")
print(f"Size: {zip_size_mb:.2f} MB")
print("="*80)
print("\nContents:")
print(f"  ✓ {saved_count} prediction JSON files")
print(f"  ✓ Model comparison table")
print(f"  ✓ Feature dataset")
print(f"  ✓ Feature descriptions")
print(f"  ✓ Validation metrics")
print(f"  ✓ Ablation study")
print("="*80)


Exporting predictions for BEST model: CV Only (Baseline)

EXPORT COMPLETE
Best Model: CV Only (Baseline)
Predictions: 600 files
ZIP File: /kaggle/working/best_model_predictions_20260128_174611.zip
Size: 3.93 MB

Contents:
  ✓ 600 prediction JSON files
  ✓ Model comparison table
  ✓ Feature dataset
  ✓ Feature descriptions
  ✓ Validation metrics
  ✓ Ablation study


## 18. Export Best Model Predictions

Create ZIP archive with predictions and evaluation metrics.

In [24]:
from datetime import datetime
import zipfile

# Export BEST model predictions to ZIP
print(f"\n{'='*80}")
print(f"CREATING ZIP ARCHIVE FOR BEST MODEL: {best_model_name}")
print(f"{'='*80}\n")

# Get predictions for best model
best_predictions = all_model_predictions[best_model_name]

# Create predictions directory
pred_dir = os.path.join(OUTPUT_DIR, "predictions")
os.makedirs(pred_dir, exist_ok=True)

# Save JSON files
saved_count = 0
for test_file, pred_data in best_predictions.items():
    output_data = {
        'image_file': pred_data['image_file'],
        'model': best_model_name,
        'data_points': pred_data['predictions']
    }
    
    output_path = os.path.join(pred_dir, test_file)
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=2)
    
    saved_count += 1

print(f"✓ Saved {saved_count} prediction JSON files")

# Create ZIP file
zip_filename = os.path.join(OUTPUT_DIR, f"cv_ml_hybrid_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add predictions
    for test_file in os.listdir(pred_dir):
        if test_file.endswith('.json'):
            file_path = os.path.join(pred_dir, test_file)
            zipf.write(file_path, arcname=f"predictions/{test_file}")
    
    # Add model comparison table
    comparison_csv = os.path.join(OUTPUT_DIR, 'final_model_comparison_600samples.csv')
    if os.path.exists(comparison_csv):
        zipf.write(comparison_csv, arcname='model_comparison_table.csv')
    
    # Add improvement analysis
    improvement_csv = os.path.join(OUTPUT_DIR, 'improvement_over_baseline_600samples.csv')
    if os.path.exists(improvement_csv):
        zipf.write(improvement_csv, arcname='improvement_analysis.csv')
    
    # Add feature dataset
    feature_csv = os.path.join(OUTPUT_DIR, 'hybrid_features_dataset.csv')
    if os.path.exists(feature_csv):
        zipf.write(feature_csv, arcname='hybrid_features_dataset.csv')
    
    # Add feature descriptions
    feature_desc = os.path.join(OUTPUT_DIR, 'feature_descriptions.csv')
    if os.path.exists(feature_desc):
        zipf.write(feature_desc, arcname='feature_descriptions.csv')
    
    # Add validation metrics
    val_metrics_file = os.path.join(OUTPUT_DIR, 'validation_metrics.csv')
    if os.path.exists(val_metrics_file):
        zipf.write(val_metrics_file, arcname='validation_metrics.csv')
    
    # Add ablation study
    ablation_file = os.path.join(OUTPUT_DIR, 'ablation_study.csv')
    if os.path.exists(ablation_file):
        zipf.write(ablation_file, arcname='ablation_study.csv')

zip_size_mb = os.path.getsize(zip_filename) / (1024 * 1024)

print(f"\n{'='*80}")
print(f"ZIP ARCHIVE CREATED SUCCESSFULLY")
print(f"{'='*80}")
print(f"Best Model: {best_model_name}")
print(f"Predictions: {saved_count} files (600 test samples)")
print(f"ZIP File: {zip_filename}")
print(f"Size: {zip_size_mb:.2f} MB")
print(f"\nArchive Contents:")
print(f"  ✓ {saved_count} prediction JSON files")
print(f"  ✓ Model comparison table (all approaches)")
print(f"  ✓ Improvement analysis vs baseline")
print(f"  ✓ Feature dataset (14 features)")
print(f"  ✓ Feature descriptions")
print(f"  ✓ Validation metrics")
print(f"  ✓ Ablation study results")
print(f"{'='*80}")
print(f"\n🎉 All results exported! Ready for submission.")


CREATING ZIP ARCHIVE FOR BEST MODEL: CV Only (Baseline)

✓ Saved 600 prediction JSON files

ZIP ARCHIVE CREATED SUCCESSFULLY
Best Model: CV Only (Baseline)
Predictions: 600 files (600 test samples)
ZIP File: /kaggle/working/cv_ml_hybrid_predictions_20260128_174612.zip
Size: 3.93 MB

Archive Contents:
  ✓ 600 prediction JSON files
  ✓ Model comparison table (all approaches)
  ✓ Improvement analysis vs baseline
  ✓ Feature dataset (14 features)
  ✓ Feature descriptions
  ✓ Validation metrics
  ✓ Ablation study results

🎉 All results exported! Ready for submission.
